In [22]:
#!pip install pandas
#!pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)


In [23]:
import os
import pandas as pd
from geopy.geocoders import Nominatim

In [17]:
# Define the folder containing the CSV files
file_path = '/home/yuna/github/CyclingVSPollution/yuna/Code/berlin_NO2_per_station.csv'  # Replace with the actual folder path

In [18]:
# Read the CSV file with semicolon delimiter
df = pd.read_csv(file_path)

In [19]:
# Convert the consolidated data into a DataFrame
df = pd.DataFrame(df)
df

,Station,NO2 Average concentration (µg/m³)
0,221 Karl-Marx-Straße,10
1,117 Schildhornstraße,9
2,042 Neukölln,3
3,282 Karlshorst,3
4,027 Marienfelde,2
5,032 Grunewald,2
6,190 Leipziger Straße,16
7,124 Mariendorfer Damm,19
8,010 Wedding,4
9,077 Buch,2


In [20]:
# Separate id and station name
# Use str.extract to separate ID and station name
df[['id', 'station_name']] = df['Station'].str.extract(r'(\d+)\s(.+)')
df.drop(columns=['Station'], inplace=True)
df = df[['id','station_name', 'NO2 Average concentration (µg/m³)']]
df

,id,station_name,NO2 Average concentration (µg/m³)
0,221,Karl-Marx-Straße,10
1,117,Schildhornstraße,9
2,042,Neukölln,3
3,282,Karlshorst,3
4,027,Marienfelde,2
5,032,Grunewald,2
6,190,Leipziger Straße,16
7,124,Mariendorfer Damm,19
8,010,Wedding,4
9,077,Buch,2


In [38]:
# Add coordinates of the stations
addresses = {
    "010" : "13353 Wedding, Amrumer Str. 1.",
    "027" : "12307 Berlin, Schichauweg 60",
    "032" : "14193 Berlin, Jagen 91",
    "042" : "12047 Berlin, Nansenstr. 10",
    "077" : "13125 Berlin, Hobrechtsfelder Chaussee 110",
    "085" : "12587 Berlin, Müggelseedamm 307",
    "117" : "12163 Berlin, Schildhornstr. 76",
    "124" : "12099 Berlin, Mariendorfer Damm 148",
    "144" : "12051 Berlin, Silbersteinstr. 5",
    "145" : "13465 Berlin, Jägerstieg 1",
    "171" : "10179 Berlin, Brückenstr. 6",
    "174" : "10247 Berlin, Frankfurter Allee 86b",
    "190" : "10117 Berlin, Leipziger Straße 5",
    "221" : "12043 Berlin, Karl-Marx-Str. 38",
    "282" : "10318 Berlin, Johanna-und-Willy-Brauer-Platz"
}

In [39]:
# Convert id to string and map addresses
df['id'] = df['id'].astype(str).str.zfill(3)  # Ensure IDs match dictionary keys with leading zeros
df['address'] = df['id'].map(addresses)
df

,id,station_name,NO2 Average concentration (µg/m³),address,latitude,longitude
0,221,Karl-Marx-Straße,10,"12043 Berlin, Karl-Marx-Str. 38",52.483752,13.429676
1,117,Schildhornstraße,9,"12163 Berlin, Schildhornstr. 76",52.463584,13.318261
2,042,Neukölln,3,"12047 Berlin, Nansenstr. 10",52.489235,13.430927
3,282,Karlshorst,3,"10318 Berlin, Johanna-und-Willy-Brauer-Platz",52.485383,13.529502
4,027,Marienfelde,2,"12307 Berlin, Schichauweg 60",52.396167,13.368787
5,032,Grunewald,2,"14193 Berlin, Jagen 91",52.492609,13.248166
6,190,Leipziger Straße,16,"10117 Berlin, Leipziger Straße 5",52.510128,13.386221
7,124,Mariendorfer Damm,19,"12099 Berlin, Mariendorfer Damm 148",52.437969,13.387570
8,010,Wedding,4,"13353 Wedding, Amrumer Str. 1.",NaN,NaN
9,077,Buch,2,"13125 Berlin, Hobrechtsfelder Chaussee 110",52.644414,13.482746


In [36]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Initialize the geocoder with a valid user agent
geolocator = Nominatim(user_agent="my_geocoder_app")

# Function to get coordinates with retry mechanism
def get_coordinates(address):
    try:
        time.sleep(1)  # Adding delay to avoid rate limits
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [40]:
# Apply geocoding function
df[['latitude', 'longitude']] = df['address'].apply(lambda x: pd.Series(get_coordinates(x)))
df

,id,station_name,NO2 Average concentration (µg/m³),address,latitude,longitude
0,221,Karl-Marx-Straße,10,"12043 Berlin, Karl-Marx-Str. 38",52.483752,13.429676
1,117,Schildhornstraße,9,"12163 Berlin, Schildhornstr. 76",52.463584,13.318261
2,042,Neukölln,3,"12047 Berlin, Nansenstr. 10",52.489235,13.430927
3,282,Karlshorst,3,"10318 Berlin, Johanna-und-Willy-Brauer-Platz",52.485383,13.529502
4,027,Marienfelde,2,"12307 Berlin, Schichauweg 60",52.396167,13.368787
5,032,Grunewald,2,"14193 Berlin, Jagen 91",52.492609,13.248166
6,190,Leipziger Straße,16,"10117 Berlin, Leipziger Straße 5",52.510128,13.386221
7,124,Mariendorfer Damm,19,"12099 Berlin, Mariendorfer Damm 148",52.437969,13.387570
8,010,Wedding,4,"13353 Wedding, Amrumer Str. 1.",52.544993,13.345580
9,077,Buch,2,"13125 Berlin, Hobrechtsfelder Chaussee 110",52.644414,13.482746


In [41]:
# Save the data
output_file = 'berlin_NO2_per_station.csv'
df.to_csv(output_file, index=False)

In [ ]:
# berlin data source: https://luftdaten.berlin.de/lqi